In [1]:
#############################################################
# 1. Libraries

import pandas as pd
import numpy as np 
import os
import glob
from tqdm import tqdm
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold
import tensorflow as tf
tf.keras.backend.clear_session()

physical_devices = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')

from tensorflow.keras import models, layers, regularizers, metrics, losses, optimizers
from tensorflow.keras.utils import Sequence

import matplotlib.pyplot as plt
import gc
import imageio
from skimage import color, io

import albumentations

#############################################################

In [2]:
#############################################################
# 2. Paths & Global Variables

## 2.1 Paths

path = '../../01_Data/'
path_sequences = path + '01_GeneratedSequences/'
path_spectograms = path + '02_GeneratedSpectograms_MelFeatures/'


df_train = pd.read_csv(path + 'train.csv')
df_sample_submission = pd.read_csv(path + 'sample_submission.csv') 

train_paths = glob.glob(path + 'train/*')
test_paths = glob.glob(path + 'test/*')

unique_segments_id_train = set(df_train['segment_id'])
unique_segments_id_test = set(df_sample_submission['segment_id'])

dict_unique_segments_id = { v : k for k, v in enumerate(unique_segments_id_train)}
dict_unique_segments_id_inv = { k : v for k, v in enumerate(unique_segments_id_train)}

## 2.2 Global Variables

SEQ_LENGTH = 60_001
IMG_SIZE = (40, 118)

#############################################################

In [3]:
#############################################################
# 3. Global Functions


def scale(x, mean_, std_):
    return (x - mean_) / std_


def unscale(x, mean_, std_):
    return (x * std_) + mean_

def getdDictsSpectoGramsNulls(dict_segments_paths):
    dict_results = {}
    for segment in tqdm(dict_segments_paths, total=len(dict_segments_paths), position=0):
        df = pd.read_csv(dict_segments_paths[segment])
        dict_results[segment] = df.isna().sum().values/SEQ_LENGTH      
    return dict_results

#############################################################

In [4]:
#############################################################
# 4. Preprocess


dict_segment_paths_train = {
    segment : path + 'train/' + str(segment) + '.csv' for segment in unique_segments_id_train
}

dict_segment_paths_test = {
    segment : path + 'test/' + str(segment) + '.csv' for segment in unique_segments_id_test
}

###

dict_segments_spectograms_paths_train = {
    segment : path_spectograms + 'train/' + str(segment) + '/'
    for segment in unique_segments_id_train
}

dict_segments_spectograms_paths_test = {
    segment : path_spectograms + 'test/' + str(segment) + '/'
    for segment in unique_segments_id_test
}

###

# mean_time_to_eruption, std_time_to_eruption = df_train['time_to_eruption'].mean(), df_train['time_to_eruption'].std()

df_train['time_to_eruption'] = df_train['time_to_eruption']/(10**6)
# df_train['time_to_eruption'] = df_train['time_to_eruption'].apply(lambda x: scale(x, mean_time_to_eruption, std_time_to_eruption))

median_time_to_eruptions = np.quantile(df_train['time_to_eruption'], 0.5)

dict_labels = {
    segment: df_train['time_to_eruption'][df_train['segment_id']==segment].values.flatten()
    for segment in unique_segments_id_train
}

###


#############################################################

In [5]:
#############################################################
# 5. Build Generator

def displayImage(image, figsize=(20, 15)):
    fig = plt.figure(figsize=figsize)
    plt.grid(False)
    plt.imshow(image)
    plt.show()


# 5.1 - Data Augmentation

def getTrainTransforms():
    return albumentations.Compose([
            albumentations.OneOf([
                albumentations.GaussNoise(p=0.2),
                albumentations.Cutout(num_holes=4, max_h_size=6, max_w_size=6, fill_value=0, p=0.2),
            ], p=0.3),
#         albumentations.OpticalDistortion(p=0.3)
#         albumentations.ShiftScaleRotate(shift_limit=0.05, rotate_limit=1, p=0.5),
#         albumentations.RandomCrop(IMG_SIZE[0]-2, IMG_SIZE[1]-5, p=0.5),
#         albumentations.PadIfNeeded(min_height=IMG_SIZE[0], min_width=IMG_SIZE[1], value=0, p=1.0)
    ])


# 5.2 - Dataset

class VolcanoSequencesGenerator(Sequence):
    
    def __init__(self, segments, path_spectograms, batch_size, dict_labels, transforms, training=True):
        super(VolcanoSequencesGenerator, self).__init__()
        
        self.segments = segments
        self.path_spectograms = path_spectograms
        self.batch_size = batch_size
        self.dict_labels = dict_labels
        self.training = training
        self.transforms = transforms
        self.on_epoch_end()
        
    def __len__(self):
        self.num_steps = int(np.ceil(len(self.segments) / self.batch_size))
        return self.num_steps
        
    def __getitem__(self, idx):
        indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        list_batch_segments = [self.segments[k] for k in indexes]
        
        array_spectograms_s0 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_0.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s1 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_1.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s2 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_2.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s3 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_3.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s4 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_4.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s5 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_5.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s6 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_6.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s7 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_7.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s8 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_8.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8)
        array_spectograms_s9 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_9.npy', 
                                                            allow_pickle=True)
                                     for segment in list_batch_segments]).astype(np.uint8) 
        
        if self.transforms:
            data_s0, data_s1 = {'image':array_spectograms_s0}, {'image':array_spectograms_s1}
            data_s2, data_s3 = {'image':array_spectograms_s2}, {'image':array_spectograms_s3}
            data_s4, data_s5 = {'image':array_spectograms_s4}, {'image':array_spectograms_s5}
            data_s6, data_s7 = {'image':array_spectograms_s6}, {'image':array_spectograms_s7}
            data_s8, data_s9 = {'image':array_spectograms_s8}, {'image':array_spectograms_s9}
            
            array_spectograms_s0 = np.stack([self.transforms(image=x)['image'] for x in data_s0['image']], axis=0)
            array_spectograms_s1 = np.stack([self.transforms(image=x)['image'] for x in data_s1['image']], axis=0)
            array_spectograms_s2 = np.stack([self.transforms(image=x)['image'] for x in data_s2['image']], axis=0)
            array_spectograms_s3 = np.stack([self.transforms(image=x)['image'] for x in data_s3['image']], axis=0)
            array_spectograms_s4 = np.stack([self.transforms(image=x)['image'] for x in data_s4['image']], axis=0)
            array_spectograms_s5 = np.stack([self.transforms(image=x)['image'] for x in data_s5['image']], axis=0)
            array_spectograms_s6 = np.stack([self.transforms(image=x)['image'] for x in data_s6['image']], axis=0)
            array_spectograms_s7 = np.stack([self.transforms(image=x)['image'] for x in data_s7['image']], axis=0)
            array_spectograms_s8 = np.stack([self.transforms(image=x)['image'] for x in data_s8['image']], axis=0)
            array_spectograms_s9 = np.stack([self.transforms(image=x)['image'] for x in data_s9['image']], axis=0)
                 
        batch = (array_spectograms_s0/255, array_spectograms_s1/255, array_spectograms_s2/255, array_spectograms_s3/255, 
                 array_spectograms_s4/255, array_spectograms_s5/255, array_spectograms_s6/255, array_spectograms_s7/255, 
                 array_spectograms_s8/255, array_spectograms_s9/255)    
            
        if self.training:
            array_labels = np.asarray([self.dict_labels[segment] for segment in list_batch_segments])
        if self.training:
            return batch, array_labels
        else:
            return batch
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.segments))
        np.random.shuffle(self.indexes)
        
        
    def generateOrderedSequences(self, list_segments):
        array_spectograms_s0 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_0.npy', 
                                                            allow_pickle=True)
                                     for segment in list_segments]).astype(np.uint8)
        array_spectograms_s1 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_1.npy', 
                                                            allow_pickle=True)
                                     for segment in list_segments]).astype(np.uint8)
        array_spectograms_s2 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_2.npy', 
                                                            allow_pickle=True)
                                     for segment in list_segments]).astype(np.uint8)
        array_spectograms_s3 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_3.npy', 
                                                            allow_pickle=True)
                                     for segment in list_segments]).astype(np.uint8)
        array_spectograms_s4 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_4.npy', 
                                                            allow_pickle=True)
                                     for segment in list_segments]).astype(np.uint8)
        array_spectograms_s5 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_5.npy', 
                                                            allow_pickle=True)
                                     for segment in list_segments]).astype(np.uint8)
        array_spectograms_s6 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_6.npy', 
                                                            allow_pickle=True)
                                     for segment in list_segments]).astype(np.uint8)
        array_spectograms_s7 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_7.npy', 
                                                            allow_pickle=True)
                                     for segment in list_segments]).astype(np.uint8)
        array_spectograms_s8 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_8.npy', 
                                                            allow_pickle=True)
                                     for segment in list_segments]).astype(np.uint8)
        array_spectograms_s9 = np.asarray([np.load(f'{self.path_spectograms[segment]}{segment}_9.npy', 
                                                            allow_pickle=True)
                                     for segment in list_segments]).astype(np.uint8)
                 
        batch = (array_spectograms_s0/255, array_spectograms_s1/255, array_spectograms_s2/255, array_spectograms_s3/255, 
                 array_spectograms_s4/255, array_spectograms_s5/255, array_spectograms_s6/255, array_spectograms_s7/255, 
                 array_spectograms_s8/255, array_spectograms_s9/255)   
        
        if self.training:
            array_labels = np.asarray([self.dict_labels[segment] for segment in list_segments])
        if self.training:
            return batch, array_labels
        else:
            return batch
    
#############################################################

In [6]:
# tmp_gen = VolcanoSequencesGenerator(list(unique_segments_id_train), dict_segments_spectograms_paths_train,
#                                                 batch_size=2, dict_labels=dict_labels, 
#                                                 transforms=getTrainTransforms(),
#                                                 training=True)

# for batch in tmp_gen:
#     break
    
# print(batch[0][0].shape)
# displayImage(batch[0][0][0], figsize=(20, 15))

In [7]:
#############################################################
# 6. Models

class ReturnBestEarlyStopping(tf.keras.callbacks.EarlyStopping):
    def __init__(self, **kwargs):
        super(ReturnBestEarlyStopping, self).__init__(**kwargs)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            if self.verbose > 0:
                print(f'\nEpoch {self.stopped_epoch + 1}: early stopping')
        elif self.restore_best_weights:
            if self.verbose > 0:
                print('Restoring model weights from the end of the best epoch.')
            self.model.set_weights(self.best_weights)     

# Custom Loss

def quantileLoss(y_true, y_pred):
    quantiles = tf.constant([0.4, 0.5, 0.6])
    e = y_true - y_pred
    v = tf.maximum(quantiles * e, (quantiles-1) * e)
    return tf.reduce_mean(v)


# Conv Model

class ConvModelSensor(models.Model):
    def __init__(self):
        super(ConvModelSensor, self).__init__()
        
        self.block0 = IdentityBlock(filters=[32, 32, 32], kernel_size=(7, 7), reg_factor=1e-4)
        self.block1 = IdentityBlock(filters=[64, 64, 64], kernel_size=(7, 7), reg_factor=1e-4)
        self.block2 = IdentityBlock(filters=[64, 64, 64], kernel_size=(5, 5), reg_factor=1e-4)
        self.block3 = IdentityBlock(filters=[128, 128, 128], kernel_size=(3, 3), reg_factor=1e-4)
        
        self.conv0 = layers.Conv2D(filters=32, kernel_size=(9, 9), 
                                  padding='same', kernel_regularizer=regularizers.l2(1e-4))
        self.bn0 = layers.BatchNormalization()
        
        self.conv1 = layers.Conv2D(filters=32, kernel_size=(7, 7), 
                                  padding='same', kernel_regularizer=regularizers.l2(1e-4))
        self.conv2 = layers.Conv2D(filters=64, kernel_size=(5, 5), 
                                  padding='same', kernel_regularizer=regularizers.l2(1e-4))
        self.conv3 = layers.Conv2D(filters=64, kernel_size=(5, 5), 
                          padding='same', kernel_regularizer=regularizers.l2(1e-4))
        self.conv4 = layers.Conv2D(filters=128, kernel_size=(3, 3), 
                          padding='same', kernel_regularizer=regularizers.l2(1e-4))
        
        self.drop0 = layers.Dropout(0.2)
        self.drop1 = layers.Dropout(0.2)
        self.drop2 = layers.Dropout(0.2)
        self.drop3 = layers.Dropout(0.2)
        
        self.max_pool0 = layers.MaxPool2D((1, 2))
        self.max_pool1 = layers.MaxPool2D(2)
        self.max_pool2 = layers.MaxPool2D(2)
        self.max_pool3 = layers.MaxPool2D(2)
        
    
    def call(self, inputs, training):
        
        x = self.conv0(inputs)
        x = self.bn0(x)
        x = tf.nn.relu(x)
        x = self.max_pool0(x)
        
        x = self.drop0(x, training=training)
        x = self.conv1(x)
        x = self.block0(x)
        x = self.max_pool1(x)
        
        x = self.drop1(x, training=training)
        x = self.conv2(x)
        x = self.block1(x)
        x = self.max_pool2(x)
        
        x = self.drop2(x, training=training)
        x = self.conv3(x)
        x = self.block2(x)
        x = self.max_pool3(x)
        
        x = self.drop3(x, training=training)
        x = self.conv4(x)
        x = self.block3(x)

        return x
    
class IdentityBlock(models.Model):
    def __init__(self, filters=[32, 32, 32], kernel_size=(3, 3), reg_factor=1e-4):
            super(IdentityBlock, self).__init__()

            self.l0 = layers.Conv2D(filters=filters[0], kernel_size=(1, 1), 
                              padding='valid', kernel_regularizer=regularizers.l2(reg_factor))
            self.l1 = layers.BatchNormalization()

            self.l2 = layers.Conv2D(filters=filters[1], kernel_size=kernel_size, 
                              padding='same', kernel_regularizer=regularizers.l2(reg_factor))
            self.l3 = layers.BatchNormalization()

            self.l4 = layers.Conv2D(filters=filters[2], kernel_size=(1, 1), 
                              padding='valid', kernel_regularizer=regularizers.l2(reg_factor))
            self.l5 = layers.BatchNormalization()
        
    def call(self, inputs, training):
        x_residual = inputs
        
        x = self.l0(inputs)
        x = self.l1(x)
        x = tf.nn.relu(x)
        
        x = self.l2(x)
        x = self.l3(x)
        x = tf.nn.relu(x)
        
        x = self.l4(x)
        x = self.l5(x)
        
        x = tf.add(x, x_residual)
        x = tf.nn.relu(x)
        
        return x
        
        

def buildModel(size, summary=False):
    
    block0 = ConvModelSensor()
    block1 = ConvModelSensor()
    block2 = ConvModelSensor()
    block3 = ConvModelSensor()
    block4 = ConvModelSensor()
    block5 = ConvModelSensor()
    block6 = ConvModelSensor()
    block7 = ConvModelSensor()
    block8 = ConvModelSensor()
    block9 = ConvModelSensor()
    
    in_0 = layers.Input(shape=(40, 118, 1))
    in_1 = layers.Input(shape=(40, 118, 1))
    in_2 = layers.Input(shape=(40, 118, 1))
    in_3 = layers.Input(shape=(40, 118, 1))
    in_4 = layers.Input(shape=(40, 118, 1))
    in_5 = layers.Input(shape=(40, 118, 1))
    in_6 = layers.Input(shape=(40, 118, 1))
    in_7 = layers.Input(shape=(40, 118, 1))
    in_8 = layers.Input(shape=(40, 118, 1))
    in_9 = layers.Input(shape=(40, 118, 1))
    
    x0 = block0(in_0)
    x1 = block1(in_1)
    x2 = block2(in_2)
    x3 = block3(in_3)
    x4 = block4(in_4)
    x5 = block5(in_5)
    x6 = block6(in_6)
    x7 = block7(in_7)
    x8 = block8(in_8)
    x9 = block9(in_9)
    
    ##################################################
    
    x0 = layers.GlobalAveragePooling2D()(x0)
    x1 = layers.GlobalAveragePooling2D()(x1)
    x2 = layers.GlobalAveragePooling2D()(x2)
    x3 = layers.GlobalAveragePooling2D()(x3)
    x4 = layers.GlobalAveragePooling2D()(x4)
    x5 = layers.GlobalAveragePooling2D()(x5)
    x6 = layers.GlobalAveragePooling2D()(x6)
    x7 = layers.GlobalAveragePooling2D()(x7)
    x8 = layers.GlobalAveragePooling2D()(x8)
    x9 = layers.GlobalAveragePooling2D()(x9)
    
    x = layers.concatenate([x0, x1, x2, x3, x4, x5, x6, x7, x8, x9])

    x = layers.Dropout(0.3)(x)
    x = layers.Dense(300, kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)
    
    ##################################################
    
    out_1 = layers.Dense(1, activation='relu', name='time_to_eruption')(x)
    out_2 = layers.Dense(3, activation='relu', name='quantile')(x)
    
    model = models.Model(inputs=[in_0, in_1, in_2, in_3, in_4,
                                 in_5, in_6, in_7, in_8, in_9], 
                         outputs=[out_1, out_2])

    model.compile(optimizer=optimizers.Adam(learning_rate=8e-4), 
                  
                  loss=[losses.MeanAbsoluteError(), quantileLoss],
                  loss_weights=[4, 1],
                  metrics=['mae'])
    if summary:
        print(model.summary())
    
    return model


def scheduler(epoch, lr):
    if epoch % 10 == 0:
        return lr*0.9
    else:
        return lr
    
    
#############################################################

In [8]:
#############################################################
# 7. Training

list_segments_train = list(unique_segments_id_train)

batch_size = 16

callback_early_stopping = ReturnBestEarlyStopping(monitor='val_time_to_eruption_loss', patience=15, 
                                                  verbose=1, restore_best_weights=True)
callback_lrsched = tf.keras.callbacks.LearningRateScheduler(scheduler)
list_callbacks = [callback_early_stopping, callback_lrsched]

kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=12)
list_history, list_models = [], []

for num_fold, (train_index, val_index) in enumerate(kf.split(list_segments_train,
                                                             np.zeros(len(list_segments_train)))):
    segments_train_fold = np.asarray(list_segments_train)[train_index]
    segments_val_fold = np.asarray(list_segments_train)[val_index]

    X_train_generator = VolcanoSequencesGenerator(segments_train_fold, dict_segments_spectograms_paths_train,
                                                  batch_size=batch_size, dict_labels=dict_labels, 
                                                  transforms=getTrainTransforms(), training=True)

    X_val_generator = VolcanoSequencesGenerator(segments_val_fold, dict_segments_spectograms_paths_train,
                                                 batch_size=batch_size, dict_labels=dict_labels, 
                                                transforms=None, training=True)

    print(f'Num Fold: {num_fold + 1}')
    print(f'Train segments: {len(train_index)} Val segments: {len(val_index)}')

    model = buildModel(size=118, summary=True)

    history = model.fit(X_train_generator,
                        validation_data=X_val_generator,
                        batch_size=batch_size,
                        callbacks=list_callbacks,
                        epochs=100,
                        verbose=1)

    list_history.append(history)
    list_models.append(model)

    X_val_batch, y_val_target = X_val_generator.generateOrderedSequences(list(segments_val_fold))
    y_pred_val = model.predict(X_val_batch, verbose=1)
    y_pred_val[0], y_pred_val[1] = y_pred_val[0]*(10**6), y_pred_val[1]*(10**6)
    y_val_target = y_val_target*(10**6)
    mae = np.abs(y_val_target - y_pred_val[0])

    model.save(f'./models/model_cnn2d_Tiny_{num_fold}')

    print('***'*20)
    print(f'Prediction MAE: {mae.mean()}')
    print('***'*20)

#############################################################

Num Fold: 1
Train segments: 3544 Val segments: 887
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40, 118, 1)] 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 40, 118, 1)] 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 40, 118, 1)] 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 40, 118, 1)] 0                                            
____________________________________

222/222 [==============================] - 39s 175ms/step - loss: 54.6949 - time_to_eruption_loss: 11.8500 - quantile_loss: 6.1173 - time_to_eruption_mae: 11.8500 - quantile_mae: 12.7191 - val_loss: 52.7729 - val_time_to_eruption_loss: 11.5025 - val_quantile_loss: 5.5665 - val_time_to_eruption_mae: 11.5025 - val_quantile_mae: 11.9128
Epoch 2/100
222/222 [==============================] - 36s 164ms/step - loss: 51.3640 - time_to_eruption_loss: 11.1497 - quantile_loss: 5.5545 - time_to_eruption_mae: 11.1497 - quantile_mae: 11.5918 - val_loss: 63.1740 - val_time_to_eruption_loss: 13.8525 - val_quantile_loss: 6.5355 - val_time_to_eruption_mae: 13.8525 - val_quantile_mae: 13.7135
Epoch 3/100
222/222 [==============================] - 37s 165ms/step - loss: 50.2415 - time_to_eruption_loss: 10.8779 - quantile_loss: 5.4884 - time_to_eruption_mae: 10.8779 - quantile_mae: 11.4318 - val_loss: 51.3486 - val_time_to_eruption_loss: 11.1327 - val_quantile_loss: 5.5669 - val_time_to_eruption_mae: 11.1

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Enric\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-50b4de7a1be3>", line 39, in <module>
    verbose=1)
  File "C:\Users\Enric\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "C:\Users\Enric\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1098, in fit
    tmp_logs = train_function(iterator)
  File "C:\Users\Enric\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\Enric\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 807, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "C:\Users\Enric\anaconda3\lib\site-packages\tensorflow\

KeyboardInterrupt: 

In [6]:
#############################################################
# 8. Cross Val Score

list_segments_train = list(unique_segments_id_train)
batch_size = 8

kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=12)
df_val_all = pd.DataFrame()

for num_fold, (train_index, val_index) in tqdm(enumerate(kf.split(list_segments_train,
                                                             np.zeros(len(list_segments_train)))), 
                                               total=5, position=0):
    
    
    segments_train_fold = np.asarray(list_segments_train)[train_index]
    segments_val_fold = np.asarray(list_segments_train)[val_index]

    X_val_generator = VolcanoSequencesGenerator(segments_val_fold, dict_segments_spectograms_paths_train,
                                                batch_size=batch_size, dict_labels=dict_labels, 
                                                transforms=None, training=True)   

    model = models.load_model(f'./models/model_cnn2d_Tiny_{num_fold}', compile=False)

    X_val_sequences, y_val_target = X_val_generator.generateOrderedSequences(segments_val_fold)
    y_pred_val = model.predict(X_val_sequences)

    df_tmp = pd.DataFrame({
            'pred' :  y_pred_val[0].squeeze()*(10**6),
            'pred_q_30' : y_pred_val[1][:, 0]*(10**6),
            'pred_q_50' : y_pred_val[1][:, 1]*(10**6),
            'pred_q_70' : y_pred_val[1][:, 2]*(10**6),
            'y_true' : y_val_target.flatten()*(10**6)
    })

    df_val_all = pd.concat([df_val_all, df_tmp], axis=0)

print('***'*20)
print(np.mean(np.abs(df_tmp['y_true'] - df_tmp['pred'])))
print(np.mean(np.abs(df_tmp['y_true'] - df_tmp['pred_q_50'])))
print('***'*20)

#############################################################

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [06:00<00:00, 72.06s/it]

************************************************************
2915986.86431257
2915679.0316216704
************************************************************


In [7]:
#############################################################
# 9. Inference


# del X_val_sequences, y_val_target, list_cv_pred, y_pred_cv, y_cv_target, df_cv, X_cv_sequences, y_cv_target
gc.collect()

list_models = [models.load_model(f'./models/model_cnn2d_Tiny_{i}', compile=False) for i in range(5)]

X_test_generator = VolcanoSequencesGenerator(list(unique_segments_id_test), dict_segments_spectograms_paths_test,
                                             batch_size=16, dict_labels=dict_labels, 
                                             transforms=None, training=False)

batch_size_prediction = 16
idx = 0
num_test_steps = int(np.ceil(len(unique_segments_id_test) / batch_size_prediction))
list_test_segments = list(unique_segments_id_test)
array_predictions = np.zeros((len(list_test_segments)))

for i in tqdm(range(num_test_steps), total=num_test_steps, position=0):
    list_tmp_segments = list_test_segments[idx:(idx+batch_size_prediction)]
    X_test_sequences = X_test_generator.generateOrderedSequences(list_tmp_segments)
     
    predictions = [model.predict(X_test_sequences)[0].squeeze() for model in list_models]
    predictions = np.mean(np.asarray(predictions), axis=0)*(10**6)
    array_predictions[idx:(idx+batch_size_prediction)] = predictions
    idx += batch_size_prediction   

df_submission = pd.DataFrame({
    'segment_id' : list_test_segments,
    'time_to_eruption' : array_predictions
})

df_submission.to_csv('./FinalSubmissions/' + 'submission_tiny_cnn.csv', index=False)
df_submission.describe()

#############################################################

  0%|                                                                                          | 0/283 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 283/283 [04:43<00:00,  1.00s/it]


,segment_id,time_to_eruption
count,4.520000e+03,4.520000e+03
mean,1.066993e+09,2.361497e+07
std,6.162904e+08,1.077405e+07
min,8.602880e+05,9.239777e+05
25%,5.458995e+08,1.624013e+07
50%,1.060695e+09,2.660927e+07
75%,1.599284e+09,3.114177e+07
max,2.147116e+09,4.847652e+07
